# 讀取評論資料和欄位處理

In [1]:
# read json file

import json
import pandas as pd

with open('./Employee Review about their organization.json') as file:
    contents = file.read()
    contents = eval(contents)
    
df_list = []
for i in range(len(contents)):
    row_list = []
    row = contents[i]
    row_list.append(row["ReviewTitle"])
    row_list.append(row["CompleteReview"])
    row_list.append(row["URL"])
    row_list.append(row["Rating"])
    row_list.append(row["ReviewDetails"])
    df_list.append(row_list)
    
df = pd.DataFrame(data = df_list, columns = ['ReviewTitle', 'CompleteReview', 'URL', 'Rating', 'ReviewDetails'])

In [2]:
# 欄位處理

df['Company'] = df.URL.str.split('/')[:].str[4]
df['isCurrentEmployee'] = df.ReviewDetails.apply(lambda x: 0 if 'Former Employee' in x else 1)
rating = df.pop('Rating')
df['Rating'] = rating
df.drop(columns = ['URL', 'ReviewDetails'], inplace = True)
df.head()

,ReviewTitle,CompleteReview,Company,isCurrentEmployee,Rating
0,Productive,"Good company, cool workplace, work load little...",Reliance-Industries-Ltd,1,3.0
1,Stressful,1. Need to work on boss's whims and fancies 2....,Reliance-Industries-Ltd,0,3.0
2,Good Company for Every employee,"Good company for every Engineers dream, Full M...",Reliance-Industries-Ltd,0,5.0
3,Productive,I am just pass out bsc in chemistry Typical da...,Reliance-Industries-Ltd,1,5.0
4,Non productive,Not so fun at work just blame games Target pe...,Reliance-Industries-Ltd,0,1.0


In [3]:
# 有 14 多萬筆評論
len(df)

145209

In [4]:
df['Rating'] = df['Rating'].astype(float)
df['Rating'].mean()

4.053660585776364

In [5]:
mean = df['Rating'].mean()
df['Rating'].iloc[df['Rating'] < mean] = 0
df['Rating'].iloc[df['Rating'] >= mean] = 1

C:\Users\ChenPoYen\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [6]:
df['Rating']

0         0.0
1         0.0
2         1.0
3         1.0
4         0.0
         ... 
145204    0.0
145205    0.0
145206    0.0
145207    0.0
145208    1.0
Name: Rating, Length: 145209, dtype: float64

In [7]:
from sklearn.model_selection import train_test_split
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

#先抽 1 %
X_other, X_train, y_other, y_train = train_test_split(X, y, test_size = 0.01, random_state = 8, stratify = y)

In [8]:
len(X_train)

1453

In [9]:
X_train['id'] = X_train.index
X_train

,ReviewTitle,CompleteReview,Company,isCurrentEmployee,id
8807,Fun Place to work and show case your skills,It's one of the best startups to work with !!\...,Ride.swiggy,1,8807
96887,Good Experience,I had a great experience working in IBM. Got e...,IBM,0,96887
65942,working profile,Pros\r Excellent work - life balance.\r Great ...,Wells-Fargo,1,65942
77935,fun to work,generating the leads\r learned how to deal wit...,Kotak-Mahindra-Bank,0,77935
88788,Kinda ok kinda not,Depends greatly on the team you go to.\r If yo...,Amazon.com,0,88788
...,...,...,...,...,...
27259,Working atmosphere and usage of modern technol...,Reliance industries is well equipped with mode...,Reliance-Industries-Ltd,0,27259
129899,Nice company to start your career,This company not only helped in learning but p...,Accenture,1,129899
120979,Good Organization,IBM is a very good organization. It is built i...,IBM,0,120979
142370,Fun-filled working environment with plenty of ...,A typical day at TCS used to be like going to ...,Tata-Consultancy-Services-(tcs),0,142370


# 文本前處理

In [10]:
#沿用 PA1 的內容
def syllables(word):
    #當字尾是e時音節數會減一
    if word.endswith("e"):
        word.replace("e", "")
    
    #當字裡包含a e i o u y時，會增加音節
    word = word.replace("a", ",")
    word = word.replace("e", ",")
    word = word.replace("i", ",")
    word = word.replace("o", ",")
    word = word.replace("u", ",")
    word = word.replace("y", ",")
    
    count = word.count(",")
    return count

In [11]:
#沿用 PA1 的內容
def tokenization(text):
    punctuation = [",", "'", ".", "?", "!", "(", ")", ":", "`", "``", "%", "{", "}", ";", "*", "#", "+", "$", 
                   "\\", "/", "_", "=", "^", "&", "<", ">", "@", "|", "[", "]"]
    for i in punctuation:
        text = text.replace(i, " ")
    
    #新增移除數字
    numbers = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0"]
    for j in numbers:
        text = text.replace(j, " ")
    
    text = text.replace("\n", " ")
    text = text.replace("\t", " ") 
    text = text.replace("\r", " ") 
    text = text.replace('"', "")
    
    #檢查 hyphens
    if "-" in text:
        t = text.count("-")
        
        #如果那個字的 hyphen 數小於2時，就會進入音節數的判斷
        if t < 2:
            count_syllables_word = text.split("-")
            s = syllables(count_syllables_word[0]) #計算第一個字音節
            
            #如果那個字的音節數小於2時，就會把字合併
            if(s < 2): 
                text = text.replace("-", "") #ex. co-worker --> coworker  
            else:
                text = text.replace("-", " ") #ex. semi-supervised --> semi supervised
        else:
            text = text.replace("-", " ") #ex. state-of-art --> state of art

    
    text = text.split(" ")
    text = list(filter(None, text)) #remove null
    
    return text

In [12]:
from nltk.stem.porter import *
import nltk
from nltk.corpus import stopwords
stemmer = PorterStemmer()
stopword_list = set(stopwords.words('english'))

docID = []
t_index = []
term = []
df = []
tf = []

for i in range(len(X_train)):
    token_list = []    

    token_list.extend(tokenization(X_train['CompleteReview'].iloc[i].encode('utf-8', 'ignore').decode('ascii', 'ignore')))
    
    #lower case
    for j in range (len(token_list)):
        if(isinstance(token_list[j], str)): #only string has to lower case 
            token_list[j] = token_list[j].lower()  
    
    #stemming
    new_token_list = [stemmer.stem(token) for token in token_list]    
    
    #stopword removal
    filtered_tokens = [token for token in new_token_list if token not in stopword_list] 
    
    #remove strange token
    filtered_tokens = [token for token in filtered_tokens if token != 'zzcadasdgadsgadgdddaddddddgggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggg']
    
    #紀錄 doc_i 中的 docID, term, tf
    for t in filtered_tokens: 
        docID.append(X_train['id'].iloc[i])
        term.append(t)
        tf.append(filtered_tokens.count(t)) #tf 的平均是用原始檔案中的字數去除

In [13]:
#step_one: 在這個步驟中將 term 跟 docID 取出
step_one = []
for i in range(len(term)):
    tmp = [term[i], str(docID[i])] #[apple, 1]
    step_one.append(tmp)

In [14]:
#step_two: 這個步驟中將 term 按照字母順序排列，並去除重複的字及記錄 df

import numpy as np
#sort alphabetically
step_two = sorted(step_one, key=lambda l:l[0])

step_two = np.array(step_two) #list to numpy

step_two = np.unique(step_two, axis=0) #去除同一個 docID 中的相同字
step_two = step_two.T
t, df = np.unique(step_two[0], return_counts=True) #計算不同檔案中的相同字數量
print(t)
print(df)

['abid' 'abil' 'abl' ... 'yr' 'zeal' 'zonal']
[ 1 11 14 ...  3  1  1]


In [15]:
my_dic ={}
my_dic["term"] = t
my_dic["df"] = df
my_dic["t_index"] = [i for i in range (1, len(t)+1)] #給定 term index (從1開始)

In [16]:
#把字典的字和對應到的文件 id 取出，方便之後找字對應到的 class
dict_ids = {}
c = 0
for t in step_two[0]:
    if t in term:
        if t not in dict_ids:
            dict_ids[t] = list()
        dict_ids[t].append(step_two[1,c])      
        
    c+=1

In [17]:
#寫入 dictionary.txt 中
dic = open("dictionary.txt", "w")
dic.write("t_index term df")
for i in range (len(my_dic["term"])):
    dic.write("\n" + str(my_dic["t_index"][i]).encode('utf-8','ignore').decode("utf-8") + " " + str(my_dic["term"][i]).encode('utf-8','ignore').decode("utf-8") + " " + str(my_dic["df"][i]).encode('utf-8','ignore').decode("utf-8"))
    
dic.close()

In [18]:
#將 docID, term, tf, t_index, df 放到同一個 dataframe，以方便計算 tf-idf
#dataframe for tf
import pandas as pd
tf_dataframe = pd.DataFrame(list(zip(docID, term, tf)), columns =['docID', 'term', 'tf'])

In [19]:
tf_dataframe

,docID,term,tf
0,8807,one,1
1,8807,best,6
2,8807,startup,1
3,8807,work,6
4,8807,good,1
...,...,...,...
32052,23390,experi,1
32053,23390,wa,1
32054,23390,good,3
32055,23390,work,2


In [20]:
#dataframe for dictionary
dict_df = pd.read_csv('dictionary.txt', delimiter = " ", keep_default_na=False)

In [21]:
#join tf_dataframe with dataframes
df3 = tf_dataframe.join(dict_df.set_index('term'), on='term')

In [22]:
df3

,docID,term,tf,t_index,df
0,8807,one,1,1968,123
1,8807,best,6,286,154
2,8807,startup,1,2663,2
3,8807,work,6,3109,1025
4,8807,good,1,1217,734
...,...,...,...,...,...
32052,23390,experi,1,997,219
32053,23390,wa,1,3033,261
32054,23390,good,3,1217,734
32055,23390,work,2,3109,1025


In [23]:
#先計算 tf-idf
N = len(X_train)
tfidf = df3['tf'] * np.log10(N / df3['df'])
df3 = df3.assign(tf_idf = tfidf)

In [24]:
#將 tf-idf 轉為 unit vector
new_df3 = df3.copy()
for i in (X_train['id']):
    #完整 vector space 大小
    matrix = df3.shape[0]   
    vector_space = np.zeros(matrix)
    rows = df3.loc[df3['docID'] == i]
    
    #取出對應的 docID 的 vector 後算距離
    vector_space[rows['t_index']] = rows['tf_idf']
    new_tfidf = rows.iloc[:, 5] / np.linalg.norm(vector_space)
    
    #改成 unit vector
    new_df3.loc[new_df3['docID'] == i, 'tf_idf'] = new_tfidf

In [25]:
new_df3

,docID,term,tf,t_index,df,tf_idf
0,8807,one,1,1968,123,0.088401
1,8807,best,6,286,154,0.482126
2,8807,startup,1,2663,2,0.235870
3,8807,work,6,3109,1025,0.074955
4,8807,good,1,1217,734,0.024448
...,...,...,...,...,...,...
32052,23390,experi,1,997,219,0.153518
32053,23390,wa,1,3033,261,0.139285
32054,23390,good,3,1217,734,0.166200
32055,23390,work,2,3109,1025,0.056617


# 整理成 TF 向量

In [26]:
# 字典的字
terms = sorted(set(new_df3['term']))
len(terms)

3148

In [27]:
# 評論 ID
doc_IDs = sorted(set(new_df3['docID']))
len(doc_IDs)

1453

In [28]:
# 轉成 TF 向量

TF_matrix = pd.DataFrame(columns = terms, index = doc_IDs)

for i in doc_IDs:
    
    terms_of_docID = new_df3[new_df3['docID'] == i].term.values
    
    for t in terms:
        if t not in terms_of_docID:
            TF_matrix.at[i, t] = 0
        else:
            TF_matrix.at[i, t] = new_df3[(new_df3['docID'] == i) & (new_df3['term'] == t)].tf.values[0]

TF_matrix

,abid,abil,abl,abov,abroad,absenc,absente,absolut,abt,abund,...,year,yearli,yet,yoga,yoir,young,youv,yr,zeal,zonal
18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
283,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
359,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
418,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
503,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144876,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
144921,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
144924,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
144989,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
